In [72]:
import DataProcess
from scipy.stats import norm
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
import optuna
from sklearn.model_selection import train_test_split
EPOCHES = 5
BATCH_SIZE = 500
NUM_OF_LAYERS = 3

In [56]:
class Model(nn.Module):
    in_features = 136
    def __init__(self, params):
        super(Model, self).__init__()
        self.layer1_linear = nn.Linear(self.in_features, params['n1'])
        self.layer1_activation = nn.ReLU()
        self.layer2_linear = nn.Linear(params['n1'], params['n2'])
        self.layer2_activation = nn.ReLU()
        self.layer3_linear = nn.Linear(params['n2'], 1)
        self.layer3_activation = nn.Sigmoid()
    def forward(self, x):
        x = self.layer1_linear(x)
        x = self.layer1_activation(x)
        x = self.layer2_linear(x)
        x = self.layer2_activation(x)
        x = self.layer3_linear(x)
        x = self.layer3_activation(x)
        return x

In [3]:
q = DataProcess.getQuery("D:\\MSLR-WEB10K\Fold1\\train.txt",10)
test_set = DataProcess.getQuery("D:\\MSLR-WEB10K\Fold1\\test.txt",10)

In [85]:
def train_and_evaluate(params):
    model = Model(params)
    model.layer1_linear.requires_grad_(requires_grad=True)
    model.layer2_linear.requires_grad_(requires_grad=True)
    model.layer3_linear.requires_grad_(requires_grad=True)
    opt = getattr(optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    #train
    for i in range(100):
        PI = q[i].claculate_pi()
        P = q[i].prob_all(PI)
        dG_ds = q[i].G_derivative_by_score(PI, P)

        # model predictions
        score = []
        dG_dw = []
        for param in model.parameters():
            dG_dw.append(torch.zeros(param.shape))
        for idx,x in enumerate(q[i].documents):
            temp = model.forward(x.feature)
            score.append(temp)
            temp.backward()
            for idx1,param in enumerate(model.parameters()):
                dG_dw[idx1] += param.grad*dG_ds[idx]/10
                param = torch.zeros(param.shape)
        for idx,param in enumerate(model.parameters()):
            param.grad = dG_dw[idx]
        q[i].score = score
        # loss

        # gradient
        # Update Parameters
        opt.zero_grad()
        opt.step()
        model.layer1_linear.weight.grad.zero_()
        model.layer1_linear.bias.grad.zero_()
        model.layer2_linear.weight.grad.zero_()
        model.layer2_linear.bias.grad.zero_()
        model.layer3_linear.weight.grad.zero_()
        model.layer3_linear.bias.grad.zero_()
    #test
    res = []
    with torch.no_grad():
        for i in range(100):
            # model predictions
            score = []
            for x in test_set[i].documents:
                temp = model.forward(x.feature)
                score.append(temp)
            test_set[i].score = score
            res.append(test_set[i].NDCG())

    print(f"Mean of Test set NDCG :{np.mean(res)*100} %")
    return np.mean(res)

In [86]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-7, 1e-1),
        'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        'n1': trial.suggest_int("n_unit", 50, 70),
        'n2': trial.suggest_int("n_unit",50,70)
    }

    meanNDCG = train_and_evaluate(params)

    return meanNDCG

In [87]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2022-12-29 01:12:49,114] A new study created in memory with name: no-name-e1c9ae15-ecfe-4b3d-a664-de5d82489b37
C:\Users\thanh\AppData\Local\Temp\ipykernel_1984\4258343626.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-7, 1e-1),
[I 2022-12-29 01:12:58,672] Trial 0 finished with value: 0.8141693536545379 and parameters: {'learning_rate': 0.00030289256042013166, 'optimizer': 'Adam', 'n_unit': 60}. Best is trial 0 with value: 0.8141693536545379.


Mean of Test set NDCG :0.8141693536545379 %


[I 2022-12-29 01:13:08,139] Trial 1 finished with value: 0.833345742360745 and parameters: {'learning_rate': 1.8616864444296277e-06, 'optimizer': 'RMSprop', 'n_unit': 61}. Best is trial 1 with value: 0.833345742360745.


Mean of Test set NDCG :0.833345742360745 %


[I 2022-12-29 01:13:18,510] Trial 2 finished with value: 0.8124454388763508 and parameters: {'learning_rate': 0.024133429242262857, 'optimizer': 'Adam', 'n_unit': 56}. Best is trial 1 with value: 0.833345742360745.


...

Mean of Test set NDCG :0.7958913819020033 %


[I 2022-12-29 01:17:22,435] Trial 28 finished with value: 0.8354519289150467 and parameters: {'learning_rate': 7.075181851800666e-05, 'optimizer': 'RMSprop', 'n_unit': 65}. Best is trial 10 with value: 0.841736078220058.


Mean of Test set NDCG :0.8354519289150467 %


[I 2022-12-29 01:17:31,573] Trial 29 finished with value: 0.792123049194 and parameters: {'learning_rate': 0.0006981896481138222, 'optimizer': 'SGD', 'n_unit': 62}. Best is trial 10 with value: 0.841736078220058.


Mean of Test set NDCG :0.792123049194 %


In [88]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 6.591195057359098e-07
optimizer: RMSprop
n_unit: 69


In [91]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_n_unit,params_optimizer,state
0,0,0.814169,2022-12-29 01:12:49.116209,2022-12-29 01:12:58.671573,0 days 00:00:09.555364,3.028926e-04,60,Adam,COMPLETE
1,1,0.833346,2022-12-29 01:12:58.672574,2022-12-29 01:13:08.139024,0 days 00:00:09.466450,1.861686e-06,61,RMSprop,COMPLETE
2,2,0.812445,2022-12-29 01:13:08.140027,2022-12-29 01:13:18.509560,0 days 00:00:10.369533,2.413343e-02,56,Adam,COMPLETE
3,3,0.814918,2022-12-29 01:13:18.510523,2022-12-29 01:13:27.730852,0 days 00:00:09.220329,5.254359e-03,58,Adam,COMPLETE
4,4,0.794671,2022-12-29 01:13:27.731793,2022-12-29 01:13:37.104012,0 days 00:00:09.372219,1.656780e-04,65,SGD,COMPLETE
5,5,0.791020,2022-12-29 01:13:37.105013,2022-12-29 01:13:46.823105,0 days 00:00:09.718092,2.902066e-03,51,RMSprop,COMPLETE
6,6,0.820652,2022-12-29 01:13:46.824103,2022-12-29 01:13:56.123332,0 days 00:00:09.299229,6.051005e-02,53,Adam,COMPLETE
7,7,0.799101,2022-12-29 01:13:56.124346,2022-12-29 01:14:05.541104,0 days 00:00:09.416758,7.736001e-03,52,Adam,COMPLETE
8,8,0.787327,2022-12-29 01:14:05.542058,2022-12-29 01:14:14.848765,0 days 00:00:09.306707,4.536391e-04,52,SGD,COMPLETE
9,9,0.810637,2022-12-29 01:14:14.849787,2022-12-29 01:14:24.094967,0 days 00:00:09.245180,4.066472e-05,64,RMSprop,COMPLETE
